# Despacho Economico - Operação

In [1]:
import numpy as np
import pandas as pd
import math as m
import cmath


HORA = 1
PB = 100

#Dados de Geração

DGER = {
    "Barra": [1,1,2,3,3],
    "UTE": [1,1,2,3,3],
    "Combustivel": [1,2,2,1,2],
    "A": [26.97,26.97,1.865,69.79,69.79],
    "B": [1.3975,3.9120,22.355,15.3116,18.66],
    "C": [0.002176,0.008176,0.001938,0.001457,0.001957],
    "GerMax":  [196/PB, 250/PB,157/PB,332/PB,388/PB],
    "GerMin":  [0,197/PB,0,0,333/PB],

}

DEMANDA = {
    "Barra": [1,2,3],
    "H1": [0,57/PB,480/PB],
}

#Dados de Linha para informar conexões elétricas

DLIN = {
    "From": [1,1,2],
    "To":   [2,3,3],
    "xkm":  [70/PB,100/PB,40/PB],
    "Limite": [200/PB,300/PB,100/PB]
}

DBAR = int(max([max(DLIN["From"]),max(DLIN["To"])]))
Ybar = np.zeros((DBAR ,DBAR)) #, dtype="complex")

In [2]:
for i in range(len(DLIN["From"])):
  f = DLIN["From"][i] -1
  to = DLIN["To"][i] -1
  y_pos = DLIN["xkm"][i]
  ykm = y_pos
  # Adicionando todas as admitancias na diagonal
  Ybar[f,f] = Ybar[f,f] + ykm
  if f != to:
    Ybar[to,to] = Ybar[to,to] + ykm
  # Adicionando fora da diagonal
    Ybar[f,to] = Ybar[f,to] - ykm
    Ybar[to,f] = Ybar[f,to]

In [3]:
df_YBAR = pd.DataFrame(Ybar, index=np.arange(1,DBAR+1), columns=np.arange(1,DBAR+1))
df_YBAR

,1,2,3
1,1.7,-0.7,-1.0
2,-0.7,1.1,-0.4
3,-1.0,-0.4,1.4


In [4]:
from pyomo.environ import *
model = ConcreteModel()

# declare decision variables
model.theta = Var(RangeSet(1,DBAR), domain=Reals, bounds=(-m.pi, m.pi))
model.PG = Var(RangeSet(1,len(DGER['UTE'])), domain=NonNegativeReals)
model.Defict = Var(RangeSet(1,DBAR), domain=NonNegativeReals)
model.AtivaGerador = Var(RangeSet(1,len(DGER['UTE'])),domain=Binary)
model.pprint()


4 RangeSet Declarations
    AtivaGerador_index : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]
    Defict_index : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    PG_index : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]
    theta_index : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]

4 Var Declarations
    AtivaGerador : Size=5, Index=AtivaGerador_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
    Defict : Size=3, Index=Defict_index
        Key : Lower : Value : Upper : Fix

In [5]:
#FuncaoObjetivo
model.profit = Objective(expr = sum(
                                    model.AtivaGerador[i]*(
                                        (DGER['A'][i-1]) 
                                        + (model.PG[i]*(DGER['B'][i-1]))
                                        + ((model.PG[i]**2)*(DGER['C'][i-1]))
                                    )
                                    + (1000*model.Defict[DGER['Barra'][i-1]])
                                    for i in model.PG_index),
                         sense=minimize)

In [6]:
#Restricao de GeracaoMax
model.GerMax = ConstraintList()
for i in range(len(DGER['GerMax'])):
  model.GerMax.add(model.PG[i+1] <= DGER['GerMax'][i])

#Restricao de GeracaoMin
model.GerMin = ConstraintList()
for i in range(len(DGER['GerMin'])):
  model.GerMin.add(model.PG[i+1] >= DGER['GerMin'][i])

In [7]:
# Restrcao Unico Combustivel
model.EscolhaCombustivel = ConstraintList()
model.EscolhaCombustivel.add(model.AtivaGerador[1] + model.AtivaGerador[2] == 1)
model.EscolhaCombustivel.add(model.AtivaGerador[4] + model.AtivaGerador[5] == 1)

In [8]:
GeracaoBarra = [ model.AtivacaoGerador[1]*model.PG[1] + model.AtivacaoGerador[2]*model.PG[2],
                 model.AtivacaoGerador[3]*model.PG[3],
                 model.AtivacaoGerador[4]* model.PG[4] + model.AtivacaoGerador[5]*model.PG[5] ] 

In [9]:
#Restricao de atendimento a Demanda

def AtendimentoDemanda(model, i):
    return sum(Ybar[i-1, j-1] * model.theta[j] for j in model.theta_index) == GeracaoBarra[i] - DEMANDA[f'H{HORA}'][i-1] + model.Defict[DGER['Barra'][i-1]]

model.AtendimentoDemanda = Constraint(model.theta_index, rule=AtendimentoDemanda)


ERROR: Rule failed when generating expression for Constraint
    AtendimentoDemanda with index 3: IndexError: list index out of range
ERROR: Constructing component 'AtendimentoDemanda' from data=None failed:
    IndexError: list index out of range


IndexError: list index out of range

In [ ]:
# Definição da Barra de Referencia
model.BarraReferencia = Constraint(expr=model.theta[1] == 0)

In [ ]:
#Restricao do Limite de Fluxo nas linhas
model.RestricaoFluxoPositivo = ConstraintList()
for i in range(len(DLIN['From'])):
    model.RestricaoFluxoPositivo.add(model.theta[DLIN['From'][i]] - model.theta[DLIN['To'][i]] <= DLIN['Limite'][i]/DLIN['xkm'][i])

model.RestricaoFluxoNegativo = ConstraintList()
for i in range(len(DLIN['From'])):
    model.RestricaoFluxoNegativo.add(-model.theta[DLIN['From'][i]] + model.theta[DLIN['To'][i]] <= DLIN['Limite'][i]/DLIN['xkm'][i])


In [ ]:
model.pprint()

In [ ]:
SolverFactory('couenne', executable='../../../scripts/couenne/couenne').solve(model).write()

# display solution
print('Custo de Geração = ', model.profit())

print('\n Thetas:')
for i in model.theta_index:
    print(f'Theta[{i}] = {value(model.theta[i])}')

print('\n Fluxo nas linhas:')
print(f"Linha 12 = {((value(model.theta[1])-value(model.theta[2]))*25)}")
print(f"Linha 13A = {((value(model.theta[1])-value(model.theta[3]))*20)}")
print(f"Linha 13B = {((value(model.theta[1])-value(model.theta[3]))*20)}")
print(f"Linha 23 = {((value(model.theta[2])-value(model.theta[3]))*20)}")

print('\n Pg:')
for i in model.PG_index:
    print(f'PG[{i}] = {value(model.PG[i])}')

print('\n Defict:')
for i in model.Defict_index:
    print(f'Defict[{i}] = {value(model.Defict[i])}')
